In [266]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyClassifier


from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report

In [267]:
df = pd.read_csv("df_train.csv", low_memory = False)

In [268]:
df.head(2)

,Unnamed: 0,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),...,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result
0,545274,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,NaN,NaN,1711,...,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,NaN,NaN,NaN,NaN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,2
1,25788,0112647160,2003-11-23,POLICE DEPARTMENT,JOHN,PEZZULLO,BROOKLYN,989.0,42.0,256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN 989.0 42.0 256 7 AVENUE BROOKLYN 1121...,BROOKLYN 359 9 AVE BROOKLYN 11215 NEW YORK,0


In [269]:
df = df.drop("Unnamed: 0", axis=1)

In [270]:
def fill_na(column):
    
    for x in column:
        df[x].fillna('UNKNOWN', inplace=True)
    
    return df.head(2)

In [271]:
column_names = list(df.columns)

In [272]:
fill_na(column_names)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),...,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result
0,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,UNKNOWN,UNKNOWN,1711,EAST 33 STREET,...,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,2
1,0112647160,2003-11-23,POLICE DEPARTMENT,JOHN,PEZZULLO,BROOKLYN,989.0,42.0,256,7 AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 989.0 42.0 256 7 AVENUE BROOKLYN 1121...,BROOKLYN 359 9 AVE BROOKLYN 11215 NEW YORK,0


In [273]:
df["Charge #2: Infraction Amount"].value_counts().head(3)

UNKNOWN    209908
0.0          2447
1000.0        157
Name: Charge #2: Infraction Amount, dtype: int64

In [274]:
#to match 

df["Respondent Address (Zip Code)"].value_counts(normalize=True)

11207    0.029952
11368    0.025825
10466    0.025820
10457    0.023809
10456    0.023218
           ...   
11703    0.000005
10385    0.000005
10066    0.000005
12182    0.000005
11167    0.000005
Name: Respondent Address (Zip Code), Length: 1108, dtype: float64

### load zip code dataframe to add neighborhood level data to the original data

In [275]:
zip_code = pd.read_csv("Demographic_Statistics_By_Zip_Code.csv", low_memory = False)

In [29]:
zip_code.loc[zip_code["JURISDICTION NAME"] == 12783]

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
229,12783,201,66,0.33,135,0.67,0,0,201,100,...,201,100,77,0.38,124,0.62,0,0,201,100


In [30]:
merged_df = pd.merge(df, zip_code, left_on = "Respondent Address (Zip Code)", right_on = "JURISDICTION NAME", how="inner")

In [31]:
merged_df['Ticket Number'].is_unique

True

In [32]:
pd.set_option('display.max_columns', 2000000000)
merged_df.iloc[277360:]

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL


In [33]:
#moved the target column to the first for easier visual

first_column = merged_df.pop('Hearing Result')
merged_df.insert(0, 'Hearing Result', first_column)
merged_df.head(2)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,2,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,UNKNOWN,UNKNOWN,1711,EAST 33 STREET,BROOKLYN,11234,NEW YORK,BROOKLYN,21,SEBA AVENUE,BROOKLYN,11229,NEW YORK,SAU: MANH,0.0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,AN68,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,11229,52,32,0.62,20,0.38,0,0,52,100,0,0.0,2,0.04,0,0.0,5,0.1,39,0.75,2,0.04,3,0.06,1,0.02,52,100,3,0.06,49,0.94,0,0.0,0,0,52,100,5,0.1,47,0.9,0,0,52,100
1,0,0176434684,2010-09-10,POLICE DEPARTMENT,KONSTANTIN,TSIPNYATOV,MANHATTAN,1016.0,36.0,1515,BROADWAY,NEW YORK,10036,NEW YORK,BROOKLYN,1815,EAST 17 STREET,BROOKLYN,11229,NEW YORK,UNKNOWN,1000.0,0.0,AG21,20-465.1,VENDING AT TIMES PLACES RESTRICTED BY RULE OF ...,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 1016.0 36.0 1515 BROADWAY NEW YORK 1...,BROOKLYN 1815 EAST 17 STREET BROOKLYN 11229 NE...,11229,52,32,0.62,20,0.38,0,0,52,100,0,0.0,2,0.04,0,0.0,5,0.1,39,0.75,2,0.04,3,0.06,1,0.02,52,100,3,0.06,49,0.94,0,0.0,0,0,52,100,5,0.1,47,0.9,0,0,52,100


In [34]:
df["Hearing Result"].value_counts(normalize=True)

0    0.429200
1    0.273505
3    0.150786
2    0.146509
Name: Hearing Result, dtype: float64

### for the respondents, some are individuals and some are commerical entities as indicated on some rows as "LLC." Therefore, need to create a separate column labeling whether the respondent is a person or otherwise

In [35]:
# pd.set_option('display.max_rows', 1000000000)
merged_df["Respondent First Name"].value_counts()

UNKNOWN      70502
MARIA         1438
JOSE          1358
LLC           1336
MOHAMED        974
             ...  
ADMEIR S         1
YANA             1
YORTY            1
PHYLLIS J        1
GUO QING         1
Name: Respondent First Name, Length: 33344, dtype: int64

In [36]:
merged_df.shape

(208022, 83)

In [37]:
key_words_first_name = ["INC", "CORP", "MANAGEMENT","BUS SERVICE AND TOUR", 
"SCIENCES DIVISION",
"HOUSING DEVELOPMENT"]  

In [38]:
key_words_last_name = ["INC", "CORP", "MANAGEMENT","FIRST HOME PROPERTIES",
"COR",
"3 NYC",
"HPENY HOUSING DEVELOPMENT FUND",
"RT HUDSON ELEMENTARY SCHOOL",
"DEVELOPMENT CO",
"HOLDING CO",
"BANANA KELLY HSG DEVE",
"AQUA PROPERTIES",
"THE BROOKLYN UNION GAS CO",
"VANDERBILT MORTGAGE AND FINANC",
"AMERICAN BROKERS CONDUIT",
"CMI BUSINESS FURNITURE",
"FRIENDS LAND DEVELOP",
"HARBOR VIEW PROP LTD",
"INGERSOLL TENANT ASSOC",
"THE BROOKLYN UNION GAS COMPANY ",
"PLAZA CONSTRUCTION",
"AUTO AUCTION"
"FIRST HOME PROP",
"1046 WASHINGTON AVE HDFC",
"DIEGO BEEKMAN MUTUAL HOUSING A",
"REV MANAGEMENT",
"LANDSLIDE PROPERTIES",
"NEIGHBORHOOD RESTORE HOUSING D",
"HTB ENTERPRISES LTD",
"ALLIANCE OF INDIVIDUA",
"WJR PROPERTIES INC",
"WJR PROPERTIES INC",
"KEYSPAN ENERGY DELIVERY NYC",
"RLTY",
"FIRST UNITED MORTGAGE BANKING",
"ASSET PLUSS MANAGEMENT SERVICE",
"KEYSPAN ENERGY DELIVERY N Y C",
"WELLS FARGO HOME MORT",
"ALLIANCE OF INDIVIDUAL",
"NEIGHBORHOOD RESTORE HDFC",
"WILMINGTON SAVINGS FUND SOCIET",
"YOUNG ISRAEL OF AVENUE K",
"FREMONT INVESTMENT LOAN",
"BELL ATLANTIC",
"EM ESS PETROLEUM CORP",
"PI CONSTRUCTION SERVICE INC",
"US BANK NATIONAL ASSOCIATION",
"CONKLIN MGMT CO",
"CON EDISON",
"CONSOLIDATED EDISON",
"EMPIRE CITY SUBWAY",
"DEUTSCHE BANK NATIONAL TRUST C",
"NATIONAL GRID",
"CONTACT HOLDINGS CORP",
"U S BANK NATIONAL ASSOCIATION",
"G G ASSOCIATES",
"WELLS FARGO BANK",
"LUCKY SEAFOOD",
"AGENT OWNER",
"FEDERAL NATIONAL MORTGAGE ASSO",
"AMENCAN HOME MORTGAGE",
"HOMESIDE LENDING",
"HSBC BANK USA",
"HSBC BANK USA NA",
"HIGH STATE RLTY CORP",
"NYC HOUSING AUTHORITY",
"PLAZA CONSTRUCTION CORP",
"EASY STREET PLUMBING INC",
"1249 WEBSTER AVE RLTY",
"DEVELOP", "BANK", "RESOURCES", "SERVICES", "LLC", "SCHOOL", "HOME","NATIONAL GRID","SAM CONEY ISLAND LLC"
                    "ALL PHASE PLUMBING CORP","ERCAT REALTY CORP"]

In [39]:
merged_df['Respondent Last Name'] = merged_df['Respondent Last Name'].astype(str)

In [40]:
# def word_checker(sentence):
#     if any(word in key_words_last_name for word in sentence.lower().split()):
#         return 'Not Person'
#     else:
#         return 'Person'

In [41]:
# merged_df['Respondent Status'] = merged_df['Respondent Last Name'].apply(word_checker)  

In [42]:
def get_word(my_string):
    for word in key_words_last_name:
        for x in merged_df["Respondent Last Name"]:
            if word.lower() in my_string.lower():
                return "Not Person"
            else:
                return "Person"

In [43]:
merged_df["Respondent Status"]= merged_df["Respondent Last Name"].apply(get_word)

In [44]:
merged_df.sample(2)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL,Respondent Status
2202,3,040623779M,2013-11-18,SANITATION POLICE,UNKNOWN,CARR JOHN,QUEENS,10002.0,22.0,95-32,150 STREET,JAMAICA,11435,NEW YORK,QUEENS,95-32,150 STREET,JAMAICA,11435,NEW YORK,UNKNOWN,300.0,280.0,AS9C,16-120 D,LOOSE RUBBISH - 2ND OFFENSE,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 10002.0 22.0 95-32 150 STREET JAMAICA 1...,QUEENS 95-32 150 STREET JAMAICA 11435 NEW YORK,11435,0,0,0.00,0,0.00,0,0,0,0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0,0,0,0,0.00,0,0.00,0,0.0,0,0,0,0,0,0.00,0,0.00,0,0,0,0,Person
77725,0,0134520110,2004-06-15,POLICE DEPARTMENT,JIE,QIU,MANHATTAN,196.0,9.0,276,CANAL STREET,MANHATTAN,10013,NEW YORK,MANHATTAN,160,MADISON STREET,MANHATTAN,10002,NEW YORK,UNKNOWN,250.0,UNKNOWN,AG02,A.C. 20-461(A),FAILURE TO CARRY AND EXHIBIT LICENSE ON DEMAND,100.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,MANHATTAN 196.0 9.0 276 CANAL STREET MANHATTAN...,MANHATTAN 160 MADISON STREET MANHATTAN 10002 N...,10002,35,19,0.54,16,0.46,0,0,35,100,0,0.0,1,0.03,0,0.0,28,0.8,6,0.17,0,0.0,0,0.0,0,0.0,35,100,2,0.06,33,0.94,0,0.0,0,0,35,100,2,0.06,33,0.94,0,0,35,100,Person


In [45]:
merged_df['Respondent Status'].value_counts()

Person        197595
Not Person     10427
Name: Respondent Status, dtype: int64

In [46]:
merged_df.loc[merged_df['Respondent Last Name'] == "NATIONAL GRID"]

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL,Respondent Status
58597,3,0182000693,2013-10-12,POLICE DEPT,UNKNOWN,NATIONAL GRID,BROOKLYN,UNKNOWN,UNKNOWN,136,SOUTH 4TH STREET,BROOKLYN,11211,NEW YORK,BROOKLYN,1,METROTECH CENTER,BROOKLYN,11201,NEW YORK,UNKNOWN,250.0,234.0,AD16,A.C. 19-122,SAND DIRT RUBBISH DEBRIS NOT REMOVED FROM SITE...,25000.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 136 SOUTH 4TH STREET BROOKLYN 11211...,BROOKLYN 1 METROTECH CENTER BROOKLYN 11201 NEW...,11201,11,6,0.55,5,0.45,0,0,11,100,0,0.0,2,0.18,0,0.0,1,0.09,1,0.09,5,0.45,2,0.18,0,0.0,11,99,0,0.0,11,1.0,0,0.0,0,0,11,100,2,0.18,9,0.82,0,0,11,100,Person
58607,1,0177039574,2010-08-09,NYPD TRANSPORT INTELLIGENCE DI,UNKNOWN,NATIONAL GRID,BROOKLYN,UNKNOWN,UNKNOWN,UNKNOWN,F O 1664 EASTERN PARKWAY BT,BROOKLYN,11233,NEW YORK,BROOKLYN,ONE,METROTECH CENTER,BROOKLYN,11201,NEW YORK,UNKNOWN,3600.0,1200.0,AD30,A.C. 19-102 II,FAILURE TO COMPLY WITH THE TERMS AND CONDITION...,1200.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN F O 1664 EASTERN PARKWAY BT BROOKL...,BROOKLYN ONE METROTECH CENTER BROOKLYN 11201 N...,11201,11,6,0.55,5,0.45,0,0,11,100,0,0.0,2,0.18,0,0.0,1,0.09,1,0.09,5,0.45,2,0.18,0,0.0,11,99,0,0.0,11,1.0,0,0.0,0,0,11,100,2,0.18,9,0.82,0,0,11,100,Person
58609,3,0182000583,2013-07-24,POLICE DEPT,UNKNOWN,NATIONAL GRID,BROOKLYN,3031.0,10.0,12,STAGG STREET,BROOKLYN,11206,NEW YORK,BROOKLYN,1,METROTECT CENTER,BROOKLYN,11201,NEW YORK,UNKNOWN,280.0,262.0,AD16,A.C. 19-122,SAND DIRT RUBBISH DEBRIS NOT REMOVED FROM SITE...,25000.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 3031.0 10.0 12 STAGG STREET BROOKLYN ...,BROOKLYN 1 METROTECT CENTER BROOKLYN 11201 NEW...,11201,11,6,0.55,5,0.45,0,0,11,100,0,0.0,2,0.18,0,0.0,1,0.09,1,0.09,5,0.45,2,0.18,0,0.0,11,99,0,0.0,11,1.0,0,0.0,0,0,11,100,2,0.18,9,0.82,0,0,11,100,Person
58610,1,0180233662,2012-04-03,NYPD TRANSPORT INTELLIGENCE DI,UNKNOWN,NATIONAL GRID

In [47]:
merged_df.drop("JURISDICTION NAME", axis=1, inplace=True)

In [48]:
merged_df.head(1)

,Hearing Result,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),Violation Location (City),Violation Location (Zip Code),Violation Location (State Name),Respondent Address (Borough),Respondent Address (House #),Respondent Address (Street Name),Respondent Address (City),Respondent Address (Zip Code),Respondent Address (State Name),Decision Location (Borough),Penalty Imposed,Paid Amount,Charge #1: Code,Charge #1: Code Section,Charge #1: Code Description,Charge #1: Infraction Amount,Charge #2: Code,Charge #2: Code Section,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,COUNT PACIFIC ISLANDER,PERCENT PACIFIC ISLANDER,COUNT HISPANIC LATINO,PERCENT HISPANIC LATINO,COUNT AMERICAN INDIAN,PERCENT AMERICAN INDIAN,COUNT ASIAN NON HISPANIC,PERCENT ASIAN NON HISPANIC,COUNT WHITE NON HISPANIC,PERCENT WHITE NON HISPANIC,COUNT BLACK NON HISPANIC,PERCENT BLACK NON HISPANIC,COUNT OTHER ETHNICITY,PERCENT OTHER ETHNICITY,COUNT ETHNICITY UNKNOWN,PERCENT ETHNICITY UNKNOWN,COUNT ETHNICITY TOTAL,PERCENT ETHNICITY TOTAL,COUNT PERMANENT RESIDENT ALIEN,PERCENT PERMANENT RESIDENT ALIEN,COUNT US CITIZEN,PERCENT US CITIZEN,COUNT OTHER CITIZEN STATUS,PERCENT OTHER CITIZEN STATUS,COUNT CITIZEN STATUS UNKNOWN,PERCENT CITIZEN STATUS UNKNOWN,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL,Respondent Status
0,2,0162420262,2012-08-30,POLICE DEPARTMENT,RONALD,BARONE,BROOKLYN,UNKNOWN,UNKNOWN,1711,EAST 33 STREET,BROOKLYN,11234,NEW YORK,BROOKLYN,21,SEBA AVENUE,BROOKLYN,11229,NEW YORK,SAU: MANH,0.0,0.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,AN68,A.C. 24-238 A,IMPROPER AUDIBLE BURGLAR ALARM WITH NO AUTOMAT...,280.0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 1711 EAST 33 STREET BROOKLYN 11234 ...,BROOKLYN 21 SEBA AVENUE BROOKLYN 11229 NEW YORK,52,32,0.62,20,0.38,0,0,52,100,0,0.0,2,0.04,0,0.0,5,0.1,39,0.75,2,0.04,3,0.06,1,0.02,52,100,3,0.06,49,0.94,0,0.0,0,0,52,100,5,0.1,47,0.9,0,0,52,100,Person


In [51]:
# pd.options.display.max_colwidth = 1000000
# pd.set_option('display.max_columns', 2000000000)
# pd.set_option('display.max_rows', 1000000000)
# pd.set_option('display.expand_frame_repr', True)

In [50]:
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(df["Violation Location (City)"], df["Hearing Result"])
y_hat = dummy_model.predict(df)

In [54]:
acc = accuracy_score(df["Hearing Result"],y_hat)
macro_precision_score=precision_score(df["Hearing Result"], y_hat, average='macro')
micro_precision_score=precision_score(df["Hearing Result"] , y_hat, average='micro')
macro_recall_score=recall_score(df["Hearing Result"], y_hat, average='macro')
micro_recall_score=recall_score(df["Hearing Result"], y_hat, average='micro')

print('Accuracy Score: {}'.format(acc))
print('Macro Precision Score: {}'.format(macro_precision_score))
print('Micro Precision Score: {}'.format(micro_precision_score))
print('Macro Recall Score: {}'.format(macro_recall_score))
print('Micro Recall Score: {}'.format(micro_recall_score))

Accuracy Score: 0.4292004895823075
Macro Precision Score: 0.10730012239557687
Micro Precision Score: 0.4292004895823075
Macro Recall Score: 0.25
Micro Recall Score: 0.4292004895823075


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Generate a column on neighborhood level income data (using Census data)

1. add a new column to the datafram with ZIP Code Tabulation Areas (ZCTAs)
2. bring in income data matching on ZCTAs
3. to download ZCTAS, will need to do web scrapping

In [177]:
nyc_zip_codes = ["10001",
"10002",
"10003",
"10004",
"10005",
"10006",
"10007",
"10009",
"10010",
"10011",
"10012",
"10013",
"10014",
"10015",
"10016",
"10017",
"10018",
"10019",
"10020",
"10021",
"10022",
"10023",
"10024",
"10025",
"10026",
"10027",
"10028",
"10029",
"10030",
"10031",
"10032",
"10033",
"10034",
"10035",
"10036",
"10037",
"10038",
"10039",
"10040",
"10041",
"10044",
"10045",
"10048",
"10055",
"10060",
"10069",
"10090",
"10095",
"10098",
"10099",
"10103",
"10104",
"10105",
"10106",
"10107",
"10110",
"10111",
"10112",
"10115",
"10118",
"10119",
"10120",
"10121",
"10122",
"10123",
"10128",
"10151",
"10152",
"10153",
"10154",
"10155",
"10158",
"10161",
"10162",
"10165",
"10166",
"10167",
"10168",
"10169",
"10170",
"10171",
"10172",
"10173",
"10174",
"10175",
"10176",
"10177",
"10178",
"10199",
"10270",
"10271",
"10278",
"10279",
"10280",
"10281",
"10282",
"10301",
"10302",
"10303",
"10304",
"10305",
"10306",
"10307",
"10308",
"10309",
"10310",
"10311",
"10312",
"10314",
"10451",
"10452",
"10453",
"10454",
"10455",
"10456",
"10457",
"10458",
"10459",
"10460",
"10461",
"10462",
"10463",
"10464",
"10465",
"10466",
"10467",
"10468",
"10469",
"10470",
"10471",
"10472",
"10473",
"10474",
"10475",
"11004",
"11101",
"11102",
"11103",
"11104",
"11105",
"11106",
"11109",
"11201",
"11203",
"11204",
"11205",
"11206",
"11207",
"11208",
"11209",
"11210",
"11211",
"11212",
"11213",
"11214",
"11215",
"11216",
"11217",
"11218",
"11219",
"11220",
"11221",
"11222",
"11223",
"11224",
"11225",
"11226",
"11228",
"11229",
"11230",
"11231",
"11232",
"11233",
"11234",
"11235",
"11236",
"11237",
"11238",
"11239",
"11241",
"11242",
"11243",
"11249",
"11252",
"11256",
"11351",
"11354",
"11355",
"11356",
"11357",
"11358",
"11359",
"11360",
"11361",
"11362",
"11363",
"11364",
"11365",
"11366",
"11367",
"11368",
"11369",
"11370",
"11371",
"11372",
"11373",
"11374",
"11375",
"11377",
"11378",
"11379",
"11385",
"11411",
"11412",
"11413",
"11414",
"11415",
"11416",
"11417",
"11418",
"11419",
"11420",
"11421",
"11422",
"11423",
"11426",
"11427",
"11428",
"11429",
"11430",
"11432",
"11433",
"11434",
"11435",
"11436",
"11691",
"11692",
"11693",
"11694",
"11697"]


In [59]:
# census_zip_codes =["00501","00544","10001","10002","10003","10004","10005","10006","10007",
#                    "10008","10009","10010","10011","10012","10013","10014","10016","10017",
#                    "10018","10019","10020","10021","10022","10023","10024","10025","10026",
#                    "10027","10028","10029","10030","10031","10032","10033","10034","10035",
#                    "10036","10037","10038","10039","10040","10041","10043","10044","10045",
#                    "10055","10060","10065","10069","10075","10080","10081","10087","10090",
#                    "10101","10102","10103","10104","10105","10106","10107","10108","10109",
#                    "10110","10111","10112","10113","10114","10115","10116","10117","10118",
#                    "10119","10120","10121","10122","10123","10124","10125","10126","10128",
#                    "10129","10130","10131","10132","10133","10138","10150","10151","10152",
#                    "10153","10154","10155","10156","10157","10158","10159","10160","10162",
#                    "10163","10164","10165","10166","10167","10168","10169","10170","10171",
#                    "10172","10173","10174","10175","10176","10177","10178","10179","10185",
#                    "10199","10203","10211","10212","10213","10242","10249","10256","10258",
#                    "10259","10260","10261","10265","10268","10269","10270","10271","10272",
#                    "10273","10274","10275","10276","10277","10278","10279","10280","10281",
#                    "10282","10285","10286","10301","10302","10303","10304","10305","10306",
#                    "10307","10308","10309","10310","10311","10312","10313","10314","10451",
#                    "10452","10453","10454","10455","10456","10457","10458","10459","10460",
#                    "10461","10462","10463","10464","10465","10466","10467","10468","10469",
#                    "10470","10471","10472","10473","10474","10475","10501","10502","10503",
#                    "10504","10505","10506","10507","10509","10510","10511","10512","10514",
#                    "10516","10517","10518","10519","10520","10521","10522","10523","10524",
#                    "10526","10527","10528","10530","10532","10533","10535","10536","10537",
#                    "10538","10540","10541","10542","10543","10545","10546","10547","10548",
#                    "10549","10550","10551","10552","10553","10560","10562","10566","10567",
#                    "10570","10573","10576","10577","10578","10579","10580","10583","10587",
#                    "10588","10589","10590","10591","10594","10595","10596","10597","10598",
#                    "10601","10602","10603","10604","10605","10606","10607","10610","10701",
#                    "10702","10703","10704","10705","10706","10707","10708","10709","10710",
#                    "10801","10802","10803","10804","10805","10901","10910","10911","10912",
#                    "10913","10914","10915","10916","10917","10918","10919","10920","10921",
#                    "10922","10923","10924","10925","10926","10927","10928","10930","10931",
#                    "10932","10933","10940","10941","10949","10950","10952","10953","10954",
#                    "10956","10958","10959","10960","10962","10963","10964","10965","10968",
#                    "10969","10970","10973","10974","10975","10976","10977","10979","10980",
#                    "10981","10982","10983","10984","10985","10986","10987","10988","10989",
#                    "10990","10992","10993","10994","10996","10997","10998","11001","11002",
#                    "11003","11004","11005","11010","11020","11021","11022","11023","11024",
#                    "11026","11027","11030","11040","11042","11050","11051","11052","11053",
#                    "11054","11055","11096","11101","11102","11103","11104","11105","11106",
#                    "11109","11120","11201","11202","11203","11204","11205","11206","11207",
#                    "11208","11209","11210","11211","11212","11213","11214","11215","11216",
#                    "11217","11218","11219","11220","11221","11222","11223","11224","11225",
#                    "11226","11228","11229","11230","11231","11232","11233","11234","11235",
#                    "11236","11237","11238","11239","11241","11242","11243","11245","11247",
#                    "11249","11251","11252","11256","11351","11352","11354","11355","11356",
#                    "11357","11358","11359","11360","11361","11362","11363","11364","11365",
#                    "11366","11367","11368","11369","11370","11371","11372","11373","11374",
#                    "11375","11377","11378","11379","11380","11381","11385","11386","11405",
#                    "11411","11412","11413","11414","11415","11416","11417","11418","11419",
#                    "11420","11421","11422","11423","11424","11425","11426","11427","11428",
#                    "11429","11430","11431","11432","11433","11434","11435","11436","11439",
#                    "11451","11499","11501","11507","11509","11510","11514","11516","11518",
#                    "11520","11530","11531","11542","11545","11547","11548","11549","11550",
#                    "11551","11552","11553","11554","11555","11556","11557","11558","11559","11560","11561",
#                    "11563","11565","11566","11568","11569","11570","11571","11572","11575","11576","11577",
#                    "11579","11580","11581","11582","11590","11596","11598","11599","11690","11691","11692",
#                    "11693","11694","11695","11697","11701","11702","11703","11704","11705","11706","11707",
#                    "11709","11710","11713","11714","11715","11716","11717","11718","11719","11720","11721",
#                    "11722","11724","11725","11726","11727","11729","11730","11731","11732","11733","11735",
#                    "11737","11738","11739","11740","11741","11742","11743","11746","11747","11749","11751",
#                    "11752","11753","11754","11755","11756","11757","11758","11760","11762","11763","11764",
#                    "11765","11766","11767","11768","11769","11770","11771","11772","11773","11775","11776",
#                    "11777","11778","11779","11780","11782","11783","11784","11786","11787","11788","11789",
#                    "11790","11791","11792","11793","11794","11795","11796","11797","11798","11801","11802",
#                    "11803","11804","11815","11853","11901","11930","11931","11932","11933","11934","11935",
#                    "11937","11939","11940","11941","11942","11944","11946","11947","11948","11949","11950",
#                    "11951","11952","11953","11954","11955","11956","11957","11958","11959","11960","11961",
#                    "11962","11963","11964","11965","11967","11968","11969","11970","11971","11972","11973",
#                    "11975","11976","11977","11978","11980","12007","12008","12009","12010","12015","12016",
#                    "12017","12018","12019","12020","12022","12023","12024","12025","12027","12028","12029",
#                    "12031","12032","12033","12035","12036","12037","12040","12041","12042","12043","12045",
#                    "12046","12047","12050","12051","12052","12053","12054","12055","12056","12057","12058",
#                    "12059","12060","12061","12062","12063","12064","12065","12066","12067","12068","12069",
#                    "12070","12071","12072","12073","12074","12075","12076","12077","12078","12082","12083",
#                    "12084","12085","12086","12087","12089","12090","12092","12093","12094","12095","12106",
#                    "12107","12108","12110","12115","12116","12117","12118","12120","12121","12122","12123",
#                    "12124","12125","12128","12130","12131","12132","12133","12134","12136","12137","12138",
#                    "12139","12140","12141","12143","12144","12147","12148","12149","12150","12151","12153",
#                    "12154","12155","12156","12157","12158","12159","12160","12161","12164","12165","12166",
#                    "12167","12168","12169","12170","12172","12173","12174","12175","12176","12177","12180",
#                    "12181","12182","12183","12184","12185","12186","12187","12188","12189","12190","12192",
#                    "12193","12194","12195","12196","12197","12198","12201","12202","12203","12204","12205",
#                    "12206","12207","12208","12209","12210","12211","12212","12214","12220","12222","12223",
#                    "12224","12225","12226","12227","12228","12229","12230","12231","12232","12233","12234",
#                    "12235","12236","12237","12238","12239","12240","12241","12242","12243","12244","12245",
#                    "12246","12247","12248","12249","12250","12255","12257","12260","12261","12288","12301",
#                    "12302","12303","12304","12305","12306","12307","12308","12309","12325","12345","12401",
#                    "12402","12404","12405","12406","12407","12409","12410","12411","12412","12413","12414",
#                    "12416","12417","12418","12419","12420","12421","12422","12423","12424","12427","12428",
#                    "12429","12430","12431","12432","12433","12434","12435","12436","12438","12439","12440",
#                    "12441","12442","12443","12444","12446","12448","12449","12450","12451","12452","12453",
#                    "12454","12455","12456","12457","12458","12459","12460","12461","12463","12464","12465",
#                    "12466","12468","12469","12470","12471","12472","12473","12474","12475","12477","12480",
#                    "12481","12482","12483","12484","12485","12486","12487","12489","12490","12491","12492",
#                    "12493","12494","12495","12496","12498","12501","12502","12503","12504","12506","12507",
#                    "12508","12510","12511","12512","12513","12514","12515","12516","12517","12518","12520",
#                    "12521","12522","12523","12524","12525","12526","12527","12528","12529","12530","12531",
#                    "12533","12534","12537","12538","12540","12541","12542","12543","12544","12545","12546","12547","12548","12549","12550","12551","12552","12553","12555","12561","12563","12564","12565","12566","12567","12568","12569","12570","12571","12572","12574","12575","12577","12578","12580","12581","12582","12583","12584","12585","12586","12588","12589","12590","12592","12594","12601","12602","12603","12604","12701","12719","12720","12721","12722","12723","12724","12725","12726","12727","12729","12732","12733","12734","12736","12737","12738","12740","12741","12742","12743","12745","12746","12747","12748","12749","12750","12751","12752","12754","12758","12759","12760","12762","12763","12764","12765","12766","12767","12768","12769","12770","12771","12775","12776","12777","12778","12779","12780","12781","12783","12784","12785","12786","12787","12788","12789","12790","12791","12792","12801","12803","12804","12808","12809","12810","12811","12812","12814","12815","12816","12817","12819","12820","12821","12822","12823","12824","12827","12828","12831","12832","12833","12834","12835","12836","12837","12838","12839","12841","12842","12843","12844","12845","12846","12847","12848","12849","12850","12851","12852","12853","12854","12855","12856","12857","12858","12859","12860","12861","12862","12863","12864","12865","12866","12870","12871","12872","12873","12874","12878","12879","12883","12884","12885","12886","12887","12901","12903","12910","12911","12912","12913","12914","12915","12916","12917","12918","12919","12920","12921","12922","12923","12924","12926","12927","12928","12929","12930","12932","12933","12934","12935","12936","12937","12939","12941","12942","12943","12944","12945","12946","12949","12950","12952","12953","12955","12956","12957","12958","12959","12960","12961","12962","12964","12965","12966","12967","12969","12970","12972","12973","12974","12975","12976","12977","12978","12979","12980","12981","12983","12985","12986","12987","12989","12992","12993","12995","12996","12997","12998","13020","13021","13022","13024","13026","13027","13028","13029","13030","13031","13032","13033","13034","13035","13036","13037","13039","13040","13041","13042","13043","13044","13045","13051","13052","13053","13054","13056","13057","13060","13061","13062","13063","13064","13065","13066","13068","13069","13071","13072","13073","13074","13076","13077","13078","13080","13081","13082","13083","13084","13087","13088","13089","13090","13092","13093","13101","13102","13103","13104","13107","13108","13110","13111","13112","13113","13114","13115","13116","13117","13118","13119","13120","13121","13122","13123","13124","13126","13131","13132","13134","13135","13136","13137","13138","13139","13140","13141","13142","13143","13144","13145","13146","13147","13148","13152","13153","13154","13155","13156","13157","13158","13159","13160","13162","13163","13164","13165","13166","13167","13201","13202","13203","13204","13205","13206","13207","13208","13209","13210","13211","13212","13214","13215","13217","13218","13219","13220","13221","13224","13225","13235","13244","13250","13251","13252","13261","13290","13301","13302","13303","13304","13305","13308","13309","13310","13312","13313","13314","13315","13316","13317","13318","13319","13320","13321","13322","13323","13324","13325","13326","13327","13328","13329","13331","13332","13333","13334","13335","13337","13338","13339","13340","13341","13342","13343","13345","13346","13348","13350","13352","13353","13354","13355","13357","13360","13361","13362","13363","13364","13365","13367","13368","13401","13402","13403","13404","13406","13407","13408","13409","13410","13411","13413","13415","13416","13417","13418","13420","13421","13424","13425","13426","13428","13431","13433","13435","13436","13437","13438","13439","13440","13441","13442","13449","13450","13452","13454","13455","13456","13457","13459","13460","13461","13464","13465","13468","13469","13470","13471","13472","13473","13475","13476","13477","13478","13479","13480","13482","13483","13484","13485","13486","13488","13489","13490","13491","13492","13493","13494","13495","13501","13502","13503","13504","13505","13599","13601","13602","13603","13605","13606","13607","13608","13611","13612","13613","13614","13615","13616","13617","13618","13619","13620","13621","13622","13623","13624","13625","13626","13627","13628","13630","13631","13632","13633","13634","13635","13636","13637","13638","13639","13640","13641","13642","13643","13645","13646","13647","13648","13649","13650","13651","13652","13654","13655","13656","13657","13658","13659","13660","13661","13662","13664","13665","13666","13667","13668","13669","13670","13671","13672","13673","13674","13675","13676","13677","13678","13679","13680","13681","13682","13683","13684","13685","13687","13690","13691","13692","13693","13694","13695","13696","13697","13699","13730","13731","13732","13733","13734","13736","13737","13738","13739","13740","13743","13744","13745","13746","13747","13748","13749","13750","13751","13752","13753","13754","13755","13756","13757","13758","13760","13761","13762","13763","13774","13775","13776","13777","13778","13780","13782","13783","13784","13786","13787","13788","13790","13794","13795","13796","13797","13801","13802","13803","13804","13806","13807","13808","13809","13810","13811","13812","13813","13814","13815","13820","13825","13826","13827","13830","13832","13833","13834","13835","13838","13839","13840","13841","13842","13843","13844","13845","13846","13847","13848","13849","13850","13851","13856","13859","13860","13861","13862","13863","13864","13865","13901","13902","13903","13904","13905","14001","14004","14005","14006","14008","14009","14010","14011","14012","14013","14020","14021","14024","14025","14026","14027","14028","14029","14030","14031","14032","14033","14034","14035","14036","14037","14038","14039","14040","14041","14042","14043","14047","14048","14051","14052","14054","14055","14056","14057","14058","14059","14060","14061","14062","14063","14065","14066","14067","14068","14069","14070","14072","14075","14080","14081","14082","14083","14085","14086","14091","14092","14094","14095","14098","14101","14102","14103","14105","14107","14108","14109","14110","14111","14112","14113","14120","14125","14126","14127","14129","14130","14131","14132","14133","14134","14135","14136","14138","14139","14140","14141","14143","14144","14145","14150","14151","14166","14167","14168","14169","14170","14171","14172","14173","14174","14201","14202","14203","14204","14205","14206","14207","14208","14209","14210","14211","14212","14213","14214","14215","14216","14217","14218","14219","14220","14221","14222","14223","14224","14225","14226","14227","14228","14231","14233","14240","14241","14260","14261","14263","14264","14265","14267","14269","14270","14272","14273","14276","14280","14301","14302","14303","14304","14305","14410","14411","14413","14414","14415","14416","14418","14420","14422","14423","14424","14425","14427","14428","14429","14430","14432","14433","14435","14437","14441","14443","14445","14449","14450","14452","14453","14454","14456","14461","14462","14463","14464","14466","14467","14468","14469","14470","14471","14472","14475","14476","14477","14478","14479","14480","14481","14482","14485","14486","14487","14488","14489","14502","14504","14505","14506","14507","14508","14510","14511","14512","14513","14514","14515","14516","14517","14518","14519","14520","14521","14522","14525","14526","14527","14529","14530","14532","14533","14534","14536","14537","14538","14539","14541","14542","14543","14544","14545","14546","14547","14548","14549","14550","14551","14555","14556","14557","14558","14559","14560","14561","14563","14564","14568","14569","14571","14572","14580","14585","14586","14588","14589","14590","14591","14592","14602","14603","14604","14605","14606","14607","14608","14609","14610","14611","14612","14613","14614","14615","14616","14617","14618","14619","14620","14621","14622","14623","14624","14625","14626","14627","14638","14639","14642","14643","14644","14646","14647","14649","14650","14651","14652","14653","14692","14694","14701","14702","14706","14707","14708","14709","14710","14711","14712","14714","14715","14716","14717","14718","14719","14720","14721","14722","14723","14724","14726","14727","14728","14729","14730","14731","14732","14733","14735","14736","14737","14738","14739","14740","14741","14742","14743","14744","14745","14747","14748","14750","14751","14752","14753","14754","14755","14756","14757","14758","14760","14766","14767","14769","14770","14772","14774","14775","14777","14778","14779","14781","14782","14783","14784","14785","14786","14787","14788","14801","14802","14803","14804","14805","14806","14807","14808","14809","14810","14812","14813","14814","14815","14816","14817","14818","14819","14820","14821","14822","14823","14824","14825","14826","14827","14830","14831","14836","14837","14838","14839","14840","14841","14842","14843","14845","14846","14847","14850","14851","14852","14853","14854","14855","14856","14857","14858","14859","14860","14861","14863","14864","14865","14867","14869","14870","14871","14872","14873","14874","14876","14877","14878","14879","14880","14881","14882","14883","14884","14885","14886","14887","14889","14891","14892","14893","14894","14895","14897","14898","14901","14902","14903","14904","14905"]

In [287]:
# this dataset contains USPS zip_code and ZCTA zip code. 
#It indicates whether some USPS and ZCTA willl match up and if it doesn't, the equivalence 
ZiptoZcta_Crosswalk_2021 = pd.read_excel("ZiptoZcta_Crosswalk_2021.xlsx")
ZiptoZcta_Crosswalk_2021["ZIP_CODE"] = ZiptoZcta_Crosswalk_2021["ZIP_CODE"].astype(str)
df['Respondent Address (Zip Code)'] = df['Respondent Address (Zip Code)'].astype(str)



# only pull out zip codes that matches the nyc zipcode list above
ZiptoZcta_Crosswalk_2021["ZIP_CODE_NYC"] = np.where(ZiptoZcta_Crosswalk_2021["ZIP_CODE"].isin(nyc_zip_codes), "NYC", "Other")

In [250]:
ZiptoZcta_Crosswalk_2021["ZIP_CODE_NYC"].value_counts()

Other    40873
NYC        234
Name: ZIP_CODE_NYC, dtype: int64

In [251]:
ZiptoZcta_Crosswalk_2021

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type,ZIP_CODE_NYC
0,10001,New York,NY,Zip Code Area,10001,Zip matches ZCTA,NYC
1,10002,New York,NY,Zip Code Area,10002,Zip matches ZCTA,NYC
2,601,Adjuntas,PR,Zip Code Area,00601,Zip matches ZCTA,Other
3,602,Aguada,PR,Zip Code Area,00602,Zip matches ZCTA,Other
4,603,Aguadilla,PR,Zip Code Area,00603,Zip matches ZCTA,Other
...,...,...,...,...,...,...,...
41102,96942,Chuuk,FM,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available",Other
41103,96943,Yap,FM,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available",Other
41104,96944,Kosrae,FM,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available",Other
41105,96960,Majuro,MH,Post Office or large volume customer,No ZCTA,"territory zip, no ZCTA available",Other


In [253]:
ZiptoZcta_Crosswalk_2021_NYC = ZiptoZcta_Crosswalk_2021.loc[ZiptoZcta_Crosswalk_2021["ZIP_CODE_NYC"] == "NYC"]
ZiptoZcta_Crosswalk_2021_NYC

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type,ZIP_CODE_NYC
0,10001,New York,NY,Zip Code Area,10001,Zip matches ZCTA,NYC
1,10002,New York,NY,Zip Code Area,10002,Zip matches ZCTA,NYC
3190,10003,New York,NY,Zip Code Area,10003,Zip matches ZCTA,NYC
3191,10004,New York,NY,Zip Code Area,10004,Zip matches ZCTA,NYC
3192,10005,New York,NY,Zip Code Area,10005,Zip matches ZCTA,NYC
...,...,...,...,...,...,...,...
5087,11242,Brooklyn,NY,Post Office or large volume customer,11201,Spatial join to ZCTA,NYC
5088,11243,Brooklyn,NY,Post Office or large volume customer,11217,Spatial join to ZCTA,NYC
5091,11249,Brooklyn,NY,Zip Code Area,11211,Spatial join to ZCTA,NYC
5093,11252,Brooklyn,NY,Zip Code Area,11209,Spatial join to ZCTA,NYC


In [255]:
ZiptoZcta_Crosswalk_2021_NYC.to_csv("NYC_Only_ZiptoZcta_Crosswalk_2021.csv")

In [257]:
ZiptoZcta_Crosswalk_2021_NYC['zip_join_type'].value_counts()

Zip matches ZCTA        205
Spatial join to ZCTA     29
Name: zip_join_type, dtype: int64

In [279]:
# get a list of the zipcode that matches ZCTA and map it onto the new column
nyc_zip_code_list = list(ZiptoZcta_Crosswalk_2021["ZIP_CODE"][ZiptoZcta_Crosswalk_2021['zip_join_type'] == "Zip matches ZCTA"])

In [278]:
df['Respondent Address (Zip Code)']

0         11229
1         11215
2         11435
3         11208
4         11367
          ...  
213238    11229
213239    10466
213240    11224
213241    11208
213242    11373
Name: Respondent Address (Zip Code), Length: 213243, dtype: int64

In [290]:
df['Respondent ZCTA'] = np.where(df['Respondent Address (Zip Code)'].isin(nyc_zip_code_list), df['Respondent Address (Zip Code)'], "Not Matched")

In [291]:
df["Respondent ZCTA"].value_counts()

11207    6387
11368    5507
10466    5506
10457    5077
10456    4951
         ... 
10172       1
10111       1
10165       1
10110       1
10271       1
Name: Respondent ZCTA, Length: 188, dtype: int64

In [292]:
df.sample(20)

,Ticket Number,Violation Date,Issuing Agency,Respondent First Name,Respondent Last Name,Violation Location (Borough),Violation Location (Block No.),Violation Location (Lot No.),Violation Location (House #),Violation Location (Street Name),...,Charge #2: Code Description,Charge #2: Infraction Amount,Charge #3: Code,Charge #3: Code Section,Charge #3: Code Description,Charge #3: Infraction Amount,complete violation location,complete respondent location,Hearing Result,Respondent ZCTA
88990,042591630N,2009-07-27,SANITATION POLICE,UNKNOWN,TAYLOR CHESTER,BRONX,2437.0,27.0,1043,COLLEGE AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BRONX 2437.0 27.0 1043 COLLEGE AVENUE BRONX 10...,BRONX 1043 COLLEGE AVENUE BRONX 10456 NEW YORK,0,10456
199991,042782655L,2016-03-15,SANITATION POLICE,UNKNOWN,1222 PUTNAM LLC,BROOKLYN,3352.0,11.0,238,PALMETTO STREET,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 3352.0 11.0 238 PALMETTO STREET BROOK...,BROOKLYN 238 PALMETTO STREET BROOKLYN 11221 NE...,1,11221
98117,046703641H,2020-01-08,SANITATION POLICE,UNKNOWN,ROLI GROUP LLC,QUEENS,12192.0,16.0,150-18,115 DRIVE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 12192.0 16.0 150-18 115 DRIVE JAMAICA 1...,QUEENS 150-18 115 DRIVE JAMAICA 11434 NEW YORK,1,11434
71777,0095590689,1998-11-19,POLICE DEPARTMENT,JUAN M,ESTRAVILLA,BRONX,UNKNOWN,UNKNOWN,400,EAST FORDHAM ROAD,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BRONX 400 EAST FORDHAM ROAD BRONX 10458 NEW ...,BRONX 2336 UNIVERSITY AVENUE BRONX 10468 NEW YORK,0,10468
56459,0133875180,2011-09-16,POLICE DEPT,YEVGENYA,IOFFE,BROOKLYN,8665.0,517.0,511,OCEAN VIEW AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 8665.0 517.0 511 OCEAN VIEW AVENUE BR...,BROOKLYN 147 NORFOLK STREET BROOKLYN 11235 NEW...,1,11235
127013,0142736853,2004-06-11,SANITATION POLICE,UNKNOWN,MANN SADICK SULLY,BROOKLYN,5236.0,24.0,628,EAST 16 STREET,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 5236.0 24.0 628 EAST 16 STREET BROOKL...,BROOKLYN 628 EAST 16 STREET BROOKLYN 11226 NEW...,3,11226
173343,046303400Y,2019-10-29,SANITATION POLICE,FARKASH,NIKOLAY,BROOKLYN,5694.0,11.0,922,57 STREET,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 5694.0 11.0 922 57 STREET BROOKLYN 11...,BROOKLYN 922 57 STREET BROOKLYN 11219 NEW YORK,2,11219
119090,0142696300,2004-06-26,SANITATION POLICE,UNKNOWN,CE MANAGEMENT LP,BRONX,2740.0,10.0,875,HUNTS POINT AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BRONX 2740.0 10.0 875 HUNTS POINT AVENUE BRONX...,BRONX 875 HUNTS POINT AVENUE BRONX 10474 NEW YORK,0,10474
109336,042770446R,2008-01-07,SANITATION POLICE,ANTHONY T,PEPE,BROOKLYN,2913.0,57.0,967,METROPOLITAN AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,BROOKLYN 2913.0 57.0 967 METROPOLITAN AVENUE B...,BROOKLYN 967 METROPOLITAN AVENUE BROOKLYN 1121...,1,11211
99605,0130352072,2002-09-28,SANITATION POLICE,UNKNOWN,WOLF PROPERTIES ASSOC,QUEENS,304.0,113.0,32-37,GREENPOINT AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,QUEENS 304.0 113.0 32-37 GREENPOINT AVENUE QUE...,QUEENS 32-37 GREENPOINT AVENUE QUEENS 11101 NE...,0,11101


In [ ]:
city_url =[]

for x in zip_codes:
    city_url.append('https://' + city +'.org/d/motorcycles-scooters/search/mca')
    
city_url